In [1]:
import pandas as pd
from rapidfuzz import process

### Using census data which is used earlier

In [3]:
df = pd.read_csv("AirQuality/RQ3/Dataset/IntermediateData_144Cities/india-districts-population-census-2011.csv")
city_state_df = pd.read_csv("AirQuality/Dataset/Ground_Truth_2023_Final.csv")
city_state_df = city_state_df[["city","state"]].drop_duplicates()

In [4]:
df.columns

Index(['District code', 'State name', 'District name', 'Population', 'Male',
       'Female', 'Literate', 'Male_Literate', 'Female_Literate', 'SC',
       ...
       'Power_Parity_Rs_90000_150000', 'Power_Parity_Rs_45000_150000',
       'Power_Parity_Rs_150000_240000', 'Power_Parity_Rs_240000_330000',
       'Power_Parity_Rs_150000_330000', 'Power_Parity_Rs_330000_425000',
       'Power_Parity_Rs_425000_545000', 'Power_Parity_Rs_330000_545000',
       'Power_Parity_Above_Rs_545000', 'Total_Power_Parity'],
      dtype='object', length=118)

In [5]:
df = df.rename(columns={
    'District name': 'city',
    'State name': 'state'
})
df['city'] = df['city'].str.strip().str.lower()
df['state'] = df['state'].str.strip().str.lower()
city_state_df['city'] = city_state_df['city'].str.strip().str.lower()
city_state_df['state'] = city_state_df['state'].str.strip().str.lower()

In [6]:
merged_df = city_state_df.merge(
    df[['city', 'state', 'Population']],
    on=['city', 'state'],
    how='left'
)

In [7]:
merged_df['Population'].isna().sum()

np.int64(89)

In [8]:
print(merged_df.columns)
print(merged_df.shape)
merged_df.to_csv("Population.csv",index=False)

Index(['city', 'state', 'Population'], dtype='object')
(201, 3)


### Using census data including town

In [11]:
city_state_df = pd.read_csv("AirQuality/Dataset/Ground_Truth_2023_Final.csv")
city_state_df = city_state_df[["city","state"]].drop_duplicates()

population_df = pd.read_excel("AirQuality/RQ3/Dataset/IntermediateData_144Cities/2011-IndiaStateDistSbDistTwn.xlsx")
population_df.columns

Index(['State', 'District', 'Subdistt', 'Town/Village', 'Ward', 'EB', 'Level',
       'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F', 'P_06', 'M_06',
       'F_06', 'P_SC', 'M_SC', 'F_SC', 'P_ST', 'M_ST', 'F_ST', 'P_LIT',
       'M_LIT', 'F_LIT', 'P_ILL', 'M_ILL', 'F_ILL', 'TOT_WORK_P', 'TOT_WORK_M',
       'TOT_WORK_F', 'MAINWORK_P', 'MAINWORK_M', 'MAINWORK_F', 'MAIN_CL_P',
       'MAIN_CL_M', 'MAIN_CL_F', 'MAIN_AL_P', 'MAIN_AL_M', 'MAIN_AL_F',
       'MAIN_HH_P', 'MAIN_HH_M', 'MAIN_HH_F', 'MAIN_OT_P', 'MAIN_OT_M',
       'MAIN_OT_F', 'MARGWORK_P', 'MARGWORK_M', 'MARGWORK_F', 'MARG_CL_P',
       'MARG_CL_M', 'MARG_CL_F', 'MARG_AL_P', 'MARG_AL_M', 'MARG_AL_F',
       'MARG_HH_P', 'MARG_HH_M', 'MARG_HH_F', 'MARG_OT_P', 'MARG_OT_M',
       'MARG_OT_F', 'MARGWORK_3_6_P', 'MARGWORK_3_6_M', 'MARGWORK_3_6_F',
       'MARG_CL_3_6_P', 'MARG_CL_3_6_M', 'MARG_CL_3_6_F', 'MARG_AL_3_6_P',
       'MARG_AL_3_6_M', 'MARG_AL_3_6_F', 'MARG_HH_3_6_P', 'MARG_HH_3_6_M',
       'MARG_HH_3_6_F', 'MARG_OT

In [12]:
city_state_df.shape

(201, 2)

In [13]:
population_df = population_df[population_df['TRU'] == 'Total'].copy()

def normalize(x):
    return str(x).strip().lower()

In [14]:
city_state_df['city'] = city_state_df['city'].apply(normalize)
city_state_df['state'] = city_state_df['state'].apply(normalize)
population_df['Name'] = population_df['Name'].apply(normalize)
population_df['Level'] = population_df['Level'].astype(str).apply(normalize)

In [15]:
state_df = population_df[population_df['Level'] == 'state']
district_df = population_df[population_df['Level'] == 'district']
subdist_df = population_df[population_df['Level'] == 'sub-district']
town_df = population_df[population_df['Level'] == 'town']

In [16]:
state_name_corrections = {
    'delhi': 'nct of delhi',
    'jammu and kashmir': 'jammu & kashmir'
}

city_state_df['state_corrected'] = city_state_df['state'].replace(state_name_corrections)

In [17]:
state_to_code = dict(zip(state_df['Name'], state_df['State']))

In [18]:
matched_rows = []

for _, row in city_state_df.iterrows():
    city = row['city']
    state = row['state_corrected']

    state_code = state_to_code.get(state)
    if state_code is None:
        matched_rows.append({
            'city': city,
            'state': state,
            'matched_city': None,
            'level': None,
            'population': None,
            'score': None
        })
        continue


    found = False
    for level_name, level_df in [
        ("district", district_df),
        ("sub-district", subdist_df),
        ("town", town_df)
    ]:
        candidates = level_df[level_df['State'] == state_code]
        if candidates.empty:
            continue

        match, score, _ = process.extractOne(city, candidates['Name'])
        if score >= 85:  # Adjust if needed
            matched_row = candidates[candidates['Name'] == match].iloc[0]
            matched_rows.append({
                'city': city,
                'state': state,
                'matched_name': match,
                'population': matched_row['TOT_P'],
                'level': level_name,
                'score': score
            })
            found = True
            break

    if not found:
        matched_rows.append({
            'city': city,
            'state': state,
            'matched_name': None,
            'population': None,
            'level': None,
            'score': None
        })


In [19]:
final_df = pd.DataFrame(matched_rows, columns=[
    'city', 'state', 'matched_city', 'level', 'population', 'score'
])

print("Total cities:", len(final_df))
print("Matched:", final_df['population'].notna().sum())
print("Unmatched:", final_df['population'].isna().sum())

Total cities: 201
Matched: 163
Unmatched: 38


In [20]:
final_df.to_csv("Population.csv",index=False)